In [1]:
# data wrangling
import pandas as pd
import numpy as np

# data viz
import matplotlib.pyplot as plt
import seaborn as sns

# parallelization
from pandarallel import pandarallel

# NLP
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Data Wrangling

In [3]:
df = pd.read_csv('../data/location_data.csv')
df

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
63690,las vegas,meadows ln,4300,89107,nevada
63691,window rock,window rock loop road,264,86515,az
63692,alliance,w state st,2496,44601,oh
63693,denver,s santa fe dr,1395,80223,co


# Exploratory Data Analysis (EDA)

In [4]:
df.nunique()

city             5534
road            17184
house_number     6353
postcode        11015
state             121
dtype: int64

In [5]:
df.sort_values(['postcode', 'city', 'road', 'house_number']).iloc[47200-5:47200+5]

,city,road,house_number,postcode,state
40363,fort worth,anderson st,8401,76120,tx
31705,fort worth,eastchase pkwy,1420,76120,texas
32066,fort worth,risinger rd,4375,76123,tx
11145,fort worth,risinger road,4375,76123,tx
54794,fort worth,risinger road,4375,76123,texas
0,fort worth,sycamore school rd,5421,76123,texas
3555,fort worth,w risinger rd,4375,76123,tx
29598,fort worth,w risinger rd,4375,76123,texas
7054,benbrook,highway 377 s,8516,76126,texas
2691,benbrook,hwy 377 s,8516,76126,tx


# Modelling - Clustering


## Distance-based clustering: tf-idf + cossine

In [6]:
flag = False
for index, col in enumerate(df.columns):
    if col == 'unique_adress':
        continue
    if not flag:
        flag=True
        df['unique_adress'] = df[col]
    else:
        df['unique_adress'] = df['unique_adress'] + ' ' + df[col] 
df['unique_adress']

0         fort worth sycamore school rd 5421 76123 texas
1              gautier highway 90 2701 39553 mississippi
2                        farmington e main 4750 87401 nm
3        richland queensgate drive 2751 99352 washington
4                  davie s state hwy seven 1600 33317 fl
                              ...                       
63690             las vegas meadows ln 4300 89107 nevada
63691     window rock window rock loop road 264 86515 az
63692                  alliance w state st 2496 44601 oh
63693                 denver s santa fe dr 1395 80223 co
63694           pearland n main st 1919 77581-3305 texas
Name: unique_adress, Length: 63695, dtype: object

In [7]:
vectorizer = TfidfVectorizer(
    stop_words=None,
    analyzer='word',
    token_pattern=r'(?<=\s)(.*?)(?=\s)' # any string between blank spaces
)
X = vectorizer.fit_transform(df['unique_adress'].tolist())

In [8]:
def print_detailed_similar_texts(text_index_of_reference, top_k=10):
    print('Text of ref: ', df['unique_adress'].tolist()[text_index_of_reference])

    cosine_similarities = linear_kernel(X[text_index_of_reference], X).flatten()
    print('Top other adresses:')

    for text_index, text_sim in zip(cosine_similarities.argsort()[:-top_k:-1], cosine_similarities[cosine_similarities.argsort()[:-top_k:-1]]):
        print(f'text index {text_index}, text similarity of {round(text_sim, 4)}:')
        print(df['unique_adress'].tolist()[text_index])
        print('\n')


text_index_of_reference = 11145 # the index "1" has an interesting example as well.
top_k = 7
print_detailed_similar_texts(text_index_of_reference, top_k)

Text of ref:  fort worth risinger road 4375 76123 tx
Top other adresses:
text index 54794, text similarity of 1.0:
fort worth risinger road 4375 76123 texas


text index 11145, text similarity of 1.0:
fort worth risinger road 4375 76123 tx


text index 32066, text similarity of 0.9584:
fort worth risinger rd 4375 76123 tx


text index 3555, text similarity of 0.9448:
fort worth w risinger rd 4375 76123 tx


text index 29598, text similarity of 0.9448:
fort worth w risinger rd 4375 76123 texas


text index 0, text similarity of 0.3838:
fort worth sycamore school rd 5421 76123 texas




In [9]:
def get_similar_texts_above_threshold(text_index_of_reference, threshold=0.9):
    cosine_similarities = linear_kernel(X[text_index_of_reference], X).flatten()
    similar_texts_indexes = np.arange(len(cosine_similarities))[cosine_similarities >= threshold]
    return similar_texts_indexes

get_similar_texts_above_threshold(11145)

array([ 3555, 11145, 29598, 32066, 54794])

In [10]:
text_groups_per_text = pd.Series(df.index.tolist()).parallel_apply(get_similar_texts_above_threshold)
text_groups_per_text

0                                        [0]
1                          [1, 14906, 19908]
2                    [2, 1552, 15581, 46925]
3                    [3, 9616, 37755, 47445]
4                            [4, 6697, 8327]
                        ...                 
63690             [840, 12445, 35566, 63690]
63691           [30354, 61541, 62662, 63691]
63692    [11710, 30912, 36891, 41429, 63692]
63693      [8791, 8801, 30752, 35584, 63693]
63694                                [63694]
Length: 63695, dtype: object

In [11]:
hash_of_text_cluster = (
    text_groups_per_text.apply(np.mean) +
    text_groups_per_text.apply(np.max) +
    text_groups_per_text.apply(np.min)
) / 3 
adresses_cluster = text_groups_per_text[~hash_of_text_cluster.duplicated()]
adresses_cluster

0                            [0]
1              [1, 14906, 19908]
2        [2, 1552, 15581, 46925]
3        [3, 9616, 37755, 47445]
4                [4, 6697, 8327]
                  ...           
63680                    [63680]
63681                    [63681]
63682                    [63682]
63685                    [63685]
63694                    [63694]
Length: 21650, dtype: object

In [12]:
def print_cluster_and_duplicated_values(cluster_id):
    cluster_values = text_groups_per_text[~hash_of_text_cluster.duplicated()][cluster_id]
    print('Texts Indexes for each value in the cluster:')
    print(cluster_values)
    print(text_groups_per_text[cluster_values])
    print()
    print('Cluster texts:')
    print(df['unique_adress'][adresses_cluster.loc[cluster_id]])

print_cluster_and_duplicated_values(5)

Texts Indexes for each value in the cluster:
[    5  6001 24115 28100 34119]
5        [5, 6001, 24115, 28100, 34119]
6001     [5, 6001, 24115, 28100, 34119]
24115    [5, 6001, 24115, 28100, 34119]
28100    [5, 6001, 24115, 28100, 34119]
34119    [5, 6001, 24115, 28100, 34119]
dtype: object

Cluster texts:
5             crestview w james lee blvd w 302 32536 fl
6001          crestview w james lee blvd w 302 32536 fl
24115      crestview w james lee blvd 302 32536 florida
28100         crestview w james lee blvd w 302 32536 fl
34119    crestview w james lee blvd w 302 32536 florida
Name: unique_adress, dtype: object


In [13]:
adresses_cluster_formatted = adresses_cluster.reset_index(
    name='record_idx'
).rename(
    columns={'index':'cluster_id'}
).assign(
    cluster_id = lambda df: np.arange(df.shape[0])
).explode('record_idx', ignore_index=True)
adresses_cluster_formatted

,cluster_id,record_idx
0,0,0
1,1,1
2,1,14906
3,1,19908
4,2,2
...,...,...
64797,21645,63680
64798,21646,63681
64799,21647,63682
64800,21648,63685


In [14]:
df_full = df.merge(
    adresses_cluster_formatted,
    left_index=True,
    right_on='record_idx'
).sort_values('cluster_id')

df_full.drop(columns='unique_adress')

,city,road,house_number,postcode,state,cluster_id,record_idx
0,fort worth,sycamore school rd,5421,76123,texas,0,0
3,gautier,hwy 90 w,2701,39553,mississippi,1,19908
2,gautier,hwy 90 w,2701,39553,ms,1,14906
1,gautier,highway 90,2701,39553,mississippi,1,1
7,farmington,e main,4750,87401,new mexico,2,46925
...,...,...,...,...,...,...,...
64797,watkins glen,e 4th st,515,14891,ny,21645,63680
64798,bremerton,wa-303,6755,98311,wa,21646,63681
64799,tampa,n dale mabry hwy,15698,33624,fl,21647,63682
64800,waterloo,university ave,2808,50701,iowa,21648,63685


## Quick overview of the model clusters

We may have one address assigned to multiple clusters. This may happen because of our modelling approach. We use the cossine similarity as our similarity function. This similarity does not obey the _triangle inequality_, meaning that if I have an specific measure of similarity between two other address, their similarity to each other is not really dictated by my similarity to them. This causes our approach to maybe assign one address to multiple clusters.

This should not be very common. In fact, when this happens, it only creates another cluster that a little bit more information that what would be the original cluster. Essentialy, we "duplicate" clusters.

One way to reduce this effect is to assign a higher threshold. Another way, would be to create another algorithm to cluster the clusters. We could use the Levenshtein distance or Jaccard distance to measure in a more detailed way the difference between clusters.

In [15]:
print('How many address are in two or more clusters?', adresses_cluster_formatted.shape[0] - df.shape[0])

How many address are in two or more clusters? 1107


In [16]:
adresses_cluster_formatted.query(
    'record_idx == 1438'
)

,cluster_id,record_idx
5370,1401,1438
5946,1552,1438


In [17]:
df_full.query(
    'cluster_id in (1401, 1552)'
)

,city,road,house_number,postcode,state,unique_adress,cluster_id,record_idx
5370,andover,central avenue,115,67002,kansas,andover central avenue 115 67002 kansas,1401,1438
5372,andover,central ave,115,67002,ks,andover central ave 115 67002 ks,1401,37271
5373,andover,central avenue,115,67002,ks,andover central avenue 115 67002 ks,1401,60275
5371,andover,central ave,115,67002,ks,andover central ave 115 67002 ks,1401,1597
5946,andover,central avenue,115,67002,kansas,andover central avenue 115 67002 kansas,1552,1438
5947,andover,central ave,115,67002,ks,andover central ave 115 67002 ks,1552,1597
5950,andover,w central ave,115,67002,kansas,andover w central ave 115 67002 kansas,1552,63478
5948,andover,central ave,115,67002,ks,andover central ave 115 67002 ks,1552,37271
5949,andover,central avenue,115,67002,ks,andover central avenue 115 67002 ks,1552,60275


In [18]:
text_groups_per_text[63478], text_groups_per_text[60275], text_groups_per_text[1438]

(array([ 1597, 37271, 63478]),
 array([ 1438,  1597, 37271, 60275]),
 array([ 1438,  1597, 37271, 60275]))

## Wrapping it up into one function

In [20]:
def approach_1_tfidf_cossine(output_path, input_path='../data/location_data.csv'):
    df = pd.read_csv(input_path)

    # creating an aggregated column
    flag = False
    for index, col in enumerate(df.columns):
        if col == 'unique_address':
            continue
        if not flag:
            flag=True
            df['unique_address'] = df[col]
        else:
            df['unique_address'] = df['unique_address'] + ' ' + df[col]

    # calculating TF-IDF for each address
    vectorizer = TfidfVectorizer(
        stop_words=None,
        analyzer='word',
        token_pattern=r'(?<=\s)(.*?)(?=\s)' # any string between blank spaces
    )
    X = vectorizer.fit_transform(df['unique_address'].tolist())

    # calculating similar addresses for each address
    def get_similar_texts_above_threshold(text_index_of_reference, threshold=0.9):
        cosine_similarities = linear_kernel(X[text_index_of_reference], X).flatten()
        similar_texts_indexes = np.arange(len(cosine_similarities))[cosine_similarities >= threshold]
        return similar_texts_indexes

    # WARNING: I'm using parallel_apply from the pandarallel package for faster performance,
    # you must activate it before using. Activate it with: "pandarallel.initialize()".
    text_groups_per_text = pd.Series(df.index.tolist()).parallel_apply(get_similar_texts_above_threshold)

    # attempting to create unique hash for each list of values of similar texts indexes.
    hash_of_text_cluster = (
        text_groups_per_text.apply(np.mean) +
        text_groups_per_text.apply(np.max) +
        text_groups_per_text.apply(np.min)
    ) / 3 
    # removing duplicated hashes, allowing only unique clusters
    adresses_cluster = text_groups_per_text[~hash_of_text_cluster.duplicated()]

    # saving data with intuitive columns to a csv format
    adresses_cluster_formatted = adresses_cluster.reset_index(
        name='record_idx'
    ).rename(
        columns={'index':'cluster_id'}
    ).assign(
        cluster_id = lambda df: np.arange(df.shape[0])
    ).explode('record_idx', ignore_index=True)

    print('How many address are in two or more clusters?', adresses_cluster_formatted.shape[0] - df.shape[0])

    df_full = df.merge(
        adresses_cluster_formatted,
        left_index=True,
        right_on='record_idx'
    ).sort_values('cluster_id')

    df_full.drop(
        columns='unique_address'
    ).to_csv(output_path, index=False)
    print('Job done!')

In [21]:
approach_1_tfidf_cossine(output_path='../data/approach_1_output.csv', input_path='../data/location_data.csv')

How many address are in two or more clusters? 1107
Job done!


In [22]:
pd.read_csv('../data/approach_1_output.csv')

,city,road,house_number,postcode,state,cluster_id,record_idx
0,fort worth,sycamore school rd,5421,76123,texas,0,0
1,gautier,hwy 90 w,2701,39553,mississippi,1,19908
2,gautier,hwy 90 w,2701,39553,ms,1,14906
3,gautier,highway 90,2701,39553,mississippi,1,1
4,farmington,e main,4750,87401,new mexico,2,46925
...,...,...,...,...,...,...,...
64797,watkins glen,e 4th st,515,14891,ny,21645,63680
64798,bremerton,wa-303,6755,98311,wa,21646,63681
64799,tampa,n dale mabry hwy,15698,33624,fl,21647,63682
64800,waterloo,university ave,2808,50701,iowa,21648,63685
